In [1]:
import numpy as np
import pandas as pd
import json
import time
from tqdm import tqdm

In [24]:
import numpy as np


EPS = 1e-8


def rot_mat(angle):
    return np.asarray([
        [np.cos(angle), -np.sin(angle)],
        [np.sin(angle), np.cos(angle)],
    ])


def point_cmp(a, b, center):
    return np.arctan2(*(a - center)[::-1]) > np.arctan2(*(b - center)[::-1])


def check_in_box2d(box, point):
    """
    :params box: (7) [x, y, z, dx, dy, dz, heading]
    """
    MARGIN = 1e-2

    # rotate the point in the opposite direction of box
    p = rot_mat(-box[6]) @ (point - box[:2])
    return (np.abs(p) < box[3:5]/2 + MARGIN).all()


def intersection(line1, line2):
    # fast exclusion: check_rect_cross
    if (
        not (line1.min(axis=0) < line2.max(axis=0)).all()
        or not (line1.max(axis=0) > line2.min(axis=0)).all()
    ):
        return None

    # check cross standing
    points = np.vstack([line1, line2])
    points_1 = points - line1[0]
    points_2 = points - line2[0]

    cross1 = np.cross(points_1[[2, 1]], points_1[[1, 3]])
    cross2 = np.cross(points_2[[0, 3]], points_2[[3, 1]])
    if cross1.prod() <= 0 or cross2.prod() <= 0:
        return None

    # calculate intersection of two lines
    # s1, s2 = cross1
    # s3, s4 = cross2
    s1 = cross1[0]
    s5 = np.cross(points_1[3], points_1[1])

    p0, p1 = line1
    q0, q1 = line2

    if abs(s5 - s1) > EPS:
        x = (s5 * q0[0] - s1 * q1[0]) / (s5 - s1)
        y = (s5 * q0[1] - s1 * q1[1]) / (s5 - s1)

    else:
        a0 = p0[1] - p1[1]
        b0 = p1[0] - p0[0]
        c0 = p0[0] * p1[1] - p1[0] * p0[1]

        a1 = q0[1] - q1[1]
        b1 = q1[0] - q0[0]
        c1 = q0[0] * q1[1] - q1[0] * q0[1]

        D = a0 * b1 - a1 * b0

        x = (b0 * c1 - b1 * c0) / D
        y = (a1 * c0 - a0 * c1) / D

    return np.array([x, y])


def box2corners(center, half_size, angle):
    corners = np.stack([-half_size, half_size], axis=0)
    corners = np.stack([
        corners[[0, 1, 1, 0], 0],
        corners[[0, 0, 1, 1], 1]
    ], axis=1)

    corners = corners @ rot_mat(angle).T + center
    return corners


def box_overlap(box_a: np.ndarray, box_b: np.ndarray):
    """
    :params box_a: [x, y, z, dx, dy, dz, heading]
    :params box_b: [x, y, z, dx, dy, dz, heading]
    """
    box_a_corners = box2corners(box_a[:2], box_a[3:5] / 2, box_a[6])
    box_b_corners = box2corners(box_b[:2], box_b[3:5] / 2, box_b[6])

    box_a_corners = np.vstack([box_a_corners, box_a_corners[:1]])
    box_b_corners = np.vstack([box_b_corners, box_b_corners[:1]])

    cnt = 0
    cross_points = np.zeros((16, 2))
    poly_center = np.zeros((2, ))
    for i in range(4):
        for j in range(4):
            cp = intersection(box_a_corners[i: i+2], box_b_corners[j: j+2])
            if cp is not None:
                cross_points[cnt] = cp
                poly_center +=  cp
                cnt += 1

    # check corners
    for k in range(4):
        if check_in_box2d(box_a, box_b_corners[k]):
            poly_center = poly_center + box_b_corners[k]
            cross_points[cnt] = box_b_corners[k]
            cnt += 1

        if check_in_box2d(box_b, box_a_corners[k]):
            poly_center = poly_center + box_a_corners[k]
            cross_points[cnt] = box_a_corners[k]
            cnt += 1

    if cnt < 3:
        assert cnt == 0
        return 0.0

    poly_center /= cnt

    # sort the points of polygon
    for j in range(cnt - 1):
        for i in range(cnt - j - 1):
            if point_cmp(cross_points[i], cross_points[i + 1], poly_center):
                cross_points[i:i+2] = cross_points[i:i+2][::-1]

    # get the overlap areas
    vectors = (cross_points[:cnt] - cross_points[0])[1:]
    area = np.cross(vectors[:-1], vectors[1:]).sum()

    return abs(area) / 2.0


def iou_bev(box_a, box_b):
    """
    :params box_a: [x, y, z, dx, dy, dz, heading]
    :params box_b: [x, y, z, dx, dy, dz, heading]
    """
    sa = box_a[3] * box_a[4]
    sb = box_b[3] * box_b[4]
    s_overlap = box_overlap(box_a, box_b)
    return s_overlap / max(sa + sb - s_overlap, EPS)


def iou_3d(boxes_a: np.ndarray, boxes_b: np.ndarray):
    """
    Args:
        boxes_a: (N, 7) [x, y, z, dx, dy, dz, heading]
        boxes_b: (M, 7) [x, y, z, dx, dy, dz, heading]

    Returns:
        ans_iou: iou, iou_a, ous_b
    """
    assert len(boxes_a) == len(boxes_b) == 7

    # height overlap
    boxes_a_height_max = (boxes_a[2] + boxes_a[5] / 2)
    boxes_a_height_min = (boxes_a[2] - boxes_a[5] / 2)
    boxes_b_height_max = (boxes_b[2] + boxes_b[5] / 2)
    boxes_b_height_min = (boxes_b[2] - boxes_b[5] / 2)

    # bev overlap
    overlaps_bev = box_overlap(boxes_a, boxes_b)

    max_of_min = max(boxes_a_height_min, boxes_b_height_min)
    min_of_max = min(boxes_a_height_max, boxes_b_height_max)
    overlaps_h = (min_of_max - max_of_min).clip(min=0)

    # 3d iou
    overlaps_3d = overlaps_bev * overlaps_h

    vol_a = (boxes_a[3] * boxes_a[4] * boxes_a[5])
    vol_b = (boxes_b[3] * boxes_b[4] * boxes_b[5])

    iou3d = overlaps_3d / (vol_a + vol_b - overlaps_3d).clip(min=1e-6)

    return iou3d, overlaps_3d / vol_a, overlaps_3d / vol_b


def whether_overlap(boxes):
    N = len(boxes)
    result = np.zeros((N, N))
    for i, box1 in enumerate(boxes):
        for j in range(i, len(boxes)):
            result[i, j] = result[j, i] = iou_3d(box1, boxes[j]) > (0.95, 0.95, 0.95)

    np.set_printoptions(suppress=True, precision=3)
    sum_r = np.sum(result)
    if sum_r <= N:
        lap = 0
    else:
        lap = 1
        # print(result)
    return lap


In [3]:
def trans_col(content):
    result = None
    if type(content) == dict:
        cur_c = list(content['center3D'].values()) + list(content['size3D'].values()) + [content['rotation3D']['z']]
        result = list(map(lambda x: round(x, 3), cur_c))
    elif type(content) == list:
        result = []
        for c in content:
            cur_c = list(c['center3D'].values()) + list(c['size3D'].values()) + [c['rotation3D']['z']]
            result.append(list(map(lambda x: round(x, 3), cur_c)))
    return result

In [4]:
# 文件路径
model_dataset = r'D:\Desktop\导出数据\model_dataset_result.csv'
model_data = r'D:\Desktop\导出数据\model_data_result.csv'
data_ann = r'D:\Desktop\导出数据\data_annotation_object.csv'

In [5]:
# 模型标注使用的列
useful_cols = ['data_id', 'model_result']
# 创建DataFrame
dataset_df = pd.read_csv(model_dataset)[useful_cols]
data_df = pd.read_csv(model_data)[useful_cols]
ann_df = pd.read_csv(data_ann)[['data_id', 'class_attributes', 'created_at', 'created_by']]

In [6]:
# 合并、整理模型表
modelrun_df = pd.concat([dataset_df, data_df]).reset_index(drop=True).dropna()
modelrun_df['model_result'] = modelrun_df['model_result'].map(lambda x: trans_col(json.loads(x).get('objects', None)))
modelrun_df = modelrun_df[modelrun_df['model_result'].map(bool)]

# 炸开
modelrun_single_df = modelrun_df.explode('model_result')

# 去重
modelrun_single_df['tag'] = modelrun_single_df['model_result'].map(lambda x: str(x))
unique_model_df = modelrun_single_df.drop_duplicates(['tag', 'data_id']).copy()
unique_model_df.rename(columns={'tag': 'model_data_tag'}, inplace=True)

In [7]:
# 整理总标注表
ann_df['class_attributes'] = ann_df['class_attributes'].map(lambda x: json.loads(x) if type(x) == str else x)
# 只需要人工处理过的数据
handle_df = ann_df[ann_df['class_attributes'].map(lambda x: 'attrs' in x)].copy()

# 区分标注类型
handle_df['type'] = handle_df['class_attributes'].map(lambda x: x['objType'])
rect_df = handle_df[handle_df['type']=='rect'].copy()
box2d_df = handle_df[handle_df['type']=='box2d'].copy()
box3d_df = handle_df[handle_df['type']=='3d'].copy()

# 对3D框去重
box3d_df['final_data'] = box3d_df['class_attributes'].map(lambda x: trans_col(x))
box3d_df['tag'] = box3d_df['final_data'].map(lambda x: str(x))
unique_3d_df = box3d_df.drop_duplicates(['tag', 'data_id']).copy()

# 去除不需要的列、改名
unique_3d_df.drop(columns=['type'], inplace=True)
unique_3d_df.rename(columns={'tag': 'final_data_tag'}, inplace=True)

In [13]:
# 统计时间
people = unique_3d_df['created_by'].unique()
for p in people:
    p_df = unique_3d_df[unique_3d_df['created_by'] == p].copy()
    p_df['time'] = p_df['created_at'].map(lambda x: time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S")))
    p_df.sort_values('time', inplace=True)
    times = p_df['time'].unique()
    total_time = 0
    start, end = 0, 0
    for i in range(1, len(times)):
        start = times[i-1]
        end = times[i]
        interval = end - start
        if interval <= 1800:
            total_time += interval
    print(p, total_time, len(p_df))

120235 65907.0 4789
120233 280501.0 10698
120238 53321.0 4020
120135 49032.0 144147
120232 87739.0 4294
120236 108418.0 5403
120240 10523.0 1038
120234 8972.0 53
120239 17981.0 1156
120241 19046.0 922
120237 98814.0 5041


In [11]:
unique_3d_df

,data_id,class_attributes,created_at,created_by,final_data,final_data_tag
13045,1998760,"{'attrs': {'TYPE': 'SUV'}, 'center3D': {'x': 9...",2022-10-25 00:53:48,120235,"[9.486, -27.979, 1.08, 4.298, 1.907, 1.3, 1.6]","[9.486, -27.979, 1.08, 4.298, 1.907, 1.3, 1.6]"
13046,1998760,"{'attrs': {'TYPE': 'SEDAN'}, 'center3D': {'x':...",2022-10-25 00:53:48,120235,"[-6.759, 25.591, 0.791, 4.343, 1.921, 1.399, -...","[-6.759, 25.591, 0.791, 4.343, 1.921, 1.399, -..."
13047,1998760,"{'attrs': {'TYPE': 'SUV'}, 'center3D': {'x': 2...",2022-10-25 00:53:48,120235,"[24.053, -19.336, 1.534, 3.967, 1.73, 1.275, 1...","[24.053, -19.336, 1.534, 3.967, 1.73, 1.275, 1..."
13048,1998760,"{'attrs': {'TYPE': 'TRUCK'}, 'center3D': {'x':...",2022-10-25 00:53:48,120235,"[14.231, -16.111, 1.711, 4.173, 2.417, 2.262, ...","[14.231, -16.111, 1.711, 4.173, 2.417, 2.262, ..."
13049,1998760,"{'attrs': {'TYPE': 'SUV'}, 'center3D': {'x': 1...",2022-10-25 00:53:48,120235,"[13.247, -19.557, 1.154, 2.739, 1.884, 1.352, ...","[13.247, -19.557, 1.154, 2.739, 1.884, 1.352, ..."
...,...,...,...,...,...,...
464048,7350324,"{'attrs': {'TYPE': 'BUS'}, 'center3D': {'x': -...",2022-11-10 15:05:24,120237,"[-40.369, 21.32, 1.128, 9.439, 2.794, 3.318, 1...","[-40.369, 21.32, 1.128, 9.439, 2.794, 3.318, 1..."
464053,7350325,"{'attrs': {'TYPE': 'SEDAN'}, 'center3D': {'x':...",2022-11-10 15:05:49,120237,"[-37.112, -22.904, 0.319, 3.767, 1.915, 1.581,...","[-37.112, -22.904, 0.319, 3.767, 1.915, 1.581,..."
464054,7350325,"{'attrs': {'TYPE': 'BUS'}, 'center3D': {'x': -...",2022-11-10 15:05:49,120237,"[-41.367, 21.531, 1.128, 9.439, 2.794, 3.318, ...","[-41.367, 21.531, 1.128, 9.439, 2.794, 3.318, ..."
464059,7350327,"{'attrs': {'TYPE': 'SEDAN'}, 'center3D': {'x':...",2022-11-10 15:06:42,120237,"[-39.015, -23.146, 0.319, 3.767, 1.915, 1.581,...","[-39.015, -23.146, 0.319, 3.767, 1.915, 1.581,..."


In [14]:
# 联表
merge_df = pd.merge(unique_3d_df, unique_model_df, how='inner', on='data_id')
# 删除不需要的列
merge_df.drop(columns=['class_attributes'], inplace=True)
# 分离出完全一样，和不完全一样的数据，完全一样的数据是客户看了，但是没动过的，可以直接算作模型的有效框
same_df = merge_df[merge_df['model_data_tag'] == merge_df['final_data_tag']]
# 获取这些框的坐标信息
same_tags = list((same_df['model_data_tag'] + same_df['data_id'].map(str)).unique())
# 模型总共标出来这么多框
print('总共:', merge_df['model_data_tag'].nunique())
# 有这么多框是动都没动直接拿去用了
print('直接用:', len(same_tags))

总共: 241675
直接用: 137996


In [15]:
# 聚合之后去除已经出现在same_tags里的框
merge_df['model_data_tag'] = merge_df['model_data_tag'] + merge_df['data_id'].map(str)
merge_df['final_data'] = merge_df['final_data'].map(lambda x: [x])
groupby_df = merge_df.groupby(by=['model_data_tag']).agg({'final_data': 'sum', 'model_result': 'first'})
diff_df = groupby_df.drop(same_tags, axis=0)
diff_df.reset_index(drop=True, inplace=True)

In [197]:
# explode_df = diff_df.explode('final_data')

In [198]:
drop_list = []
for idx, cur_row in explode_df.iterrows():
    # if cur_row['final_data']

,final_data,model_result
0,"[28.713, 14.584, 0.214, 3.7, 2.068, 1.17, -2.672]","[-0.001, 23.083, 0.983, 2.782, 2.633, 4.133, -..."
0,"[27.228, 5.872, 0.966, 5.473, 2.082, 2.103, -1...","[-0.001, 23.083, 0.983, 2.782, 2.633, 4.133, -..."
1,"[-24.965, 0.33, 0.502, 2.204, 1.779, 1.086, -0...","[-0.004, 22.158, 0.729, 2.37, 1.603, 1.522, 1...."
2,"[-20.444, -0.073, 0.475, 2.042, 1.965, 1.174, ...","[-0.005, 16.393, 0.499, 4.585, 1.808, 1.354, 3..."
2,"[30.315, -0.386, 0.593, 4.419, 2.324, 1.627, -...","[-0.005, 16.393, 0.499, 4.585, 1.808, 1.354, 3..."
...,...,...
103706,"[-43.248, 3.207, 0.384, 4.26, 1.797, 1.301, 0....","[9.998, 3.488, 0.924, 4.323, 1.915, 1.792, 0.015]"
103707,"[4.322, -2.217, 0.851, 5.035, 2.054, 1.98, 0.409]","[9.999, -27.328, -0.098, 1.966, 0.604, 0.862, ..."
103707,"[-11.248, -4.731, 0.649, 4.681, 1.971, 1.392, ...","[9.999, -27.328, -0.098, 1.966, 0.604, 0.862, ..."
103707,"[-13.376, 1.089, 0.924, 5.727, 2.203, 1.605, 0...","[9.999, -27.328, -0.098, 1.966, 0.604, 0.862, ..."


In [25]:
# m被用上的模型框
m = 0
iou_count = 0
for _, cur_row in diff_df.iterrows():
    # 总标注结果：双层列表[[7元素], [7元素], ...]
    total_list = cur_row['final_data']
    # 模型结果：7元素列表
    model_result = cur_row['model_result']

    # 调用计算IOU的函数
    for box1 in total_list:
        boxes = [np.array(box1), np.array(model_result)]
        lap = whether_overlap(boxes)
        if lap:
            iou_count += 1
        else:
            continue
print(iou_count)

AssertionError: 

In [18]:
model_result

[-0.001, 23.083, 0.983, 2.782, 2.633, 4.133, -2.149]

In [23]:
iou_3d(np.array(box1), np.array(model_result))

(0.0, 0.0, 0.0)

In [19]:
box1

[28.713, 14.584, 0.214, 3.7, 2.068, 1.17, -2.672]